In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
INPUT_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Indirect Lending\Adhoc_Pricing_Disparity_20250822\output\output.parquet")

In [ ]:
df = pd.read_parquet(INPUT_PATH)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df = df.drop(columns={
    'Loan Paid or Open',
    'Date Closed'
}).copy()

In [ ]:
from deltalake import DeltaTable

In [ ]:
ACCOUNT_DATA = Path(r"C:\Users\w322800\Documents\lakehouse\silver\account")

In [ ]:
active_accounts = DeltaTable(ACCOUNT_DATA).to_pandas()

In [ ]:
active_accounts

In [ ]:
active_accounts = active_accounts[(active_accounts['Category'].isin(['Indirect'])) | (active_accounts['currmiaccttypcd']).isin(['CM15','CM16'])].copy()

In [ ]:
df.info()

In [ ]:
active_accounts_slice = active_accounts[[
    'acctnbr',
    'ownersortname',
    'contractdate',
    'curracctstatcd',
    'noteopenamt',
    'notebal',
    'noteintrate'
]].copy()

In [ ]:
active_accounts_slice = active_accounts_slice.rename(columns={
    'acctnbr': 'Account Number',
    'contractdate': 'Loan Origination Date',
    'ownersortname': 'Applicant Last Name', # Needs splitting
    'noteopenamt': 'Amount Financed',
    'notebal': 'Current Balance',
    'noteintrate': 'Contract Rate',
}).copy()

In [ ]:
active_accounts_slice.info()

In [ ]:
merged_df = pd.merge(df, active_accounts_slice, on='Account Number', how='outer', suffixes=('_df','_active'), indicator=True)

In [ ]:
merged_df

In [ ]:
merged_df.info()

In [ ]:
merged_df

In [ ]:
import numpy as np

In [ ]:
# Your Status column is perfect. It correctly identifies loans present
# in the active_accounts table as 'Active'.
merged_df['Status'] = np.where(merged_df['_merge'] == 'left_only', 'Closed','Active')
# Note charged off are included in closed (as they are no longer active accounts)

merged_df['Contract Rate'] = merged_df['Contract Rate_df']

merged_df['Current Rate'] = np.where(
    merged_df['Status'] == 'Active',
    merged_df['Contract Rate_active'],
    np.nan
)

merged_df['Current Balance'] = np.where(
    merged_df['Status'] == 'Active',
    merged_df['Current Balance_active'],
    0
)

# %%
# Define the base names of the columns that were duplicated during the merge
cols_to_consolidate = [
    'Loan Origination Date',
    'Applicant Last Name',
    'Amount Financed',
    # 'Current Balance',
    # 'Contract Rate'
]

# Consolidate the columns using combine_first
# This method is ideal for this scenario. It takes values from the first
# DataFrame (`_active` columns) and fills any missing (NaN) values with
# data from the second DataFrame (`_df` columns).
# This perfectly matches your logic: "If it's active, use the active data...
# otherwise it should be the left dataframe."
for col in cols_to_consolidate:
    active_col = f'{col}_active'
    df_col = f'{col}_df'
    merged_df[col] = merged_df[active_col].combine_first(merged_df[df_col])

# %%
# Create a list of the original suffixed columns to drop, plus the merge indicator
cols_to_drop = [f'{col}_{suffix}' for col in cols_to_consolidate for suffix in ['df', 'active']]
cols_to_drop.append('_merge')

# Create the final, clean DataFrame
final_df = merged_df.drop(columns=cols_to_drop)
final_df = final_df.drop(columns=['Current Balance_df','Current Balance_active','curracctstatcd','Contract Rate_df','Contract Rate_active']).copy()


# %%
# As you noted, the 'Applicant Last Name' came from 'ownersortname'
# and may need to be split if the format is 'LAST, FIRST'.
# final_df['Applicant Last Name'] = final_df['Applicant Last Name'].str.split(',').str[0].str.strip()

# %%
# final_df is now your fully consolidated DataFrame
final_df.info()

# %%
# Verify the status distribution
print("\nLoan Status Counts:")
print(final_df['Status'].value_counts())

# %%
# Display the head of the final DataFrame
final_df


In [ ]:
from datetime import datetime

In [ ]:
final_df = final_df[(final_df['Loan Origination Date'] <= datetime(2024, 12, 31)) & (final_df['Loan Origination Date'] >= datetime(2020, 1, 1))].copy()

In [ ]:
final_df

In [ ]:
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore
from typing import Optional
from datetime import datetime, timedelta
import pandas as pd

def fetch_data(specified_date: Optional[datetime] = None):
    """
    Main data query
    """
    oracle_sql_query = f"""
    WITH CalculatedValues AS (
    SELECT
        a.ACCTNBR,
        a.EFFDATE,
        -- Get the first non-null rate by ordering time from oldest to newest
        FIRST_VALUE(a.NOTEINTRATE IGNORE NULLS) OVER (
        PARTITION BY a.ACCTNBR
        ORDER BY a.EFFDATE ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS first_noteintrate,

        -- Get the latest tax number by ordering time from oldest to newest
        LAST_VALUE(a.TAXRPTFORORGNBR) OVER (
        PARTITION BY a.ACCTNBR 
        ORDER BY a.EFFDATE ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS latest_taxrptfororgnbr,

        -- Get the latest tax number by ordering time from oldest to newest
        LAST_VALUE(a.TAXRPTFORPERSNBR) OVER (
        PARTITION BY a.ACCTNBR 
        ORDER BY a.EFFDATE ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS latest_taxrptforpersnbr
    FROM
        COCCDM.WH_ACCTCOMMON a -- <<< Replace with your actual table name
    )
    SELECT DISTINCT
    acctnbr,
    first_noteintrate,
    latest_taxrptfororgnbr,
    latest_taxrptforpersnbr
    FROM
    CalculatedValues
    ORDER BY
    acctnbr
    """

    queries = [
        {'key':'oracle_sql_query', 'sql':text(oracle_sql_query), 'engine':2},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
data = fetch_data()

In [ ]:
historical_data = data['oracle_sql_query'].copy()

In [ ]:
historical_data

In [ ]:
historical_data.info()

In [ ]:
historical_data['Account Number'] = historical_data['acctnbr'].astype(str)

In [ ]:
def float_to_int_string(value):
    if pd.isna(value):
        return None
    return str(int(value))

In [ ]:
historical_data['taxrptfororgnbr'] = historical_data['latest_taxrptfororgnbr'].apply(float_to_int_string)
historical_data['taxrptforpersnbr'] = historical_data['latest_taxrptforpersnbr'].apply(float_to_int_string)


In [ ]:
historical_data = historical_data[[
    'Account Number',
    'taxrptfororgnbr',
    'taxrptforpersnbr',
    'first_noteintrate'
]].copy()

In [ ]:
merge_final_hist = pd.merge(final_df, historical_data, on='Account Number', how='left')

In [ ]:
merge_final_hist

In [ ]:
merge_final_hist['Contract Rate'] = np.where((merge_final_hist['Contract Rate']).isnull(), merge_final_hist['first_noteintrate'], merge_final_hist['Contract Rate'])

In [ ]:
merge_final_hist.info()

In [ ]:
merge_final_hist = merge_final_hist.drop(columns=['first_noteintrate']).copy()

In [ ]:
merge_final_hist['Contract Rate'] = pd.to_numeric(merge_final_hist['Contract Rate'])

In [ ]:
merge_final_hist['Year Originated'] = merge_final_hist['Loan Origination Date'].dt.year

In [ ]:
merge_final_hist = merge_final_hist.rename(columns={'Applicant Last Name':'Customer Name'}).copy()

In [ ]:
merge_final_hist

In [ ]:
FINAL_OUTPUT = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Indirect Lending\Adhoc_Pricing_Disparity_20250822\output\output2.parquet")
merge_final_hist.to_parquet(FINAL_OUTPUT, index=False)

In [ ]:
allroles = DeltaTable(Path(r"C:\Users\w322800\Documents\lakehouse\bronze\wh_allroles")).to_pandas()

In [ ]:
allroles